In [ ]:
pip install wget

In [129]:
import astropy
from astropy.table import Table,vstack, Column
from subprocess import call
import subprocess
from astropy.io import fits
import numpy as np
import wget
import matplotlib.pyplot as plt
from gpz_pype.utilities import Params, set_gpz_path, basic_lupt_soft, flux_to_lupt
import astropy.units as u
from numpy.ma import masked
from astropy.coordinates import SkyCoord
from astropy.coordinates import match_coordinates_sky

from os import listdir
from os.path import isfile, join

In [ ]:
fields= ['ceers-full','gdn','gds','ngdeep','primer-cosmos-east','primer-cosmos-west','primer-uds-north','primer-uds-south','abell2744clu','abells1063','elgordo','j112716p4228','macs0647','rxj2129','smacs0723','sunrise']

# Download Field Files


In [ ]:
for field in fields:
    url=f'https://s3.amazonaws.com/grizli-v2/JwstMosaics/v7/{field}-grizli-v7.0_phot.fits'
    filename=wget.download(url)
    print(f' {field} downloaded')

# Initial Catalogue Making - Run Once


In [ ]:
###INITIAL CATALOGUE MAKING
hdu_list = fits.open('MPhys/DAWN_JWST/ceers-full-grizli-v7.0_phot.fits', memmap=True,formats='fits')
ceers = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/gdn-grizli-v7.0_phot.fits', memmap=True,formats='fits')
gdn= Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/gds-grizli-v7.0_phot.fits', memmap=True,formats='fits')
gds = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/ngdeep-grizli-v7.0_phot.fits', memmap=True,formats='fits')
ngdeep = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/primer-cosmos-east-grizli-v7.0_phot.fits', memmap=True,formats='fits')
cosmos_east = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/primer-cosmos-west-grizli-v7.0_phot.fits', memmap=True,formats='fits')
cosmos_west = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/primer-uds-south-grizli-v7.0_phot.fits', memmap=True,formats='fits')
udss = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/primer-uds-north-grizli-v7.0_phot.fits', memmap=True,formats='fits')
udsn = Table(hdu_list[1].data)
hdu_list.close()

In [ ]:
#ADDING FIELDS TO CATALOGUE - RUN BOTH CELLS IF STARTING FROM SCRATCH
hdu_list = fits.open('MPhys/DAWN_JWST/abell2744clu-grizli-v7.0_phot.fits', memmap=True,formats='fits')
abell2744 = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/abells1063-grizli-v7.0_phot.fits', memmap=True,formats='fits')
abells1063= Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/elgordo-grizli-v7.0_phot.fits', memmap=True,formats='fits')
elgordo = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/j112716p4228-grizli-v7.0_phot.fits', memmap=True,formats='fits')
j_field = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/macs0647-grizli-v7.0_phot.fits', memmap=True,formats='fits')
macs = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/rxj2129-grizli-v7.0_phot.fits', memmap=True,formats='fits')
rxj = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/smacs0723-grizli-v7.0_phot.fits', memmap=True,formats='fits')
smacs = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/sunrise-grizli-v7.0_phot.fits', memmap=True,formats='fits')
sunrise = Table(hdu_list[1].data)
hdu_list.close()

In [ ]:
abell2744.add_column('abells2744clu', name='field', index=0)
abells1063.add_column('abells1063', name='field', index=0)
elgordo.add_column('elgordo', name='field', index=0)
j_field.add_column('j112716p4228', name='field', index=0)
macs.add_column('macs0647', name='field', index=0)
rxj.add_column('rxj2129', name='field', index=0)
smacs.add_column('smacs0723', name='field', index=0)
sunrise.add_column('sunrise', name='field', index=0)

In [ ]:
ceers.add_column('ceers', name='field', index=0)
gdn.add_column('gdn', name='field', index=0)
gds.add_column('gds', name='field', index=0)
ngdeep.add_column('ngdeep', name='field', index=0)
cosmos_east.add_column('cosmos_east', name='field', index=0)
cosmos_west.add_column('cosmos_west', name='field', index=0)
udsn.add_column('udsn', name='field', index=0)
udss.add_column('udss', name='field', index=0)

In [ ]:
fields_new= ['ceers','gdn','gds','ngdeep','cosmos_east','cosmos_west','udsn','udss','abell2744','abells1063','elgordo','j_field','macs','rxj','smacs','sunrise']

# Convert to Luptiudes for Each Field


In [ ]:
field=sunrise


In [ ]:
filters0 = [col.split('_')[0] for col in field.colnames if col.endswith('flux_aper_0')]
filters0=filters0[1:]
filters1 = [col.split('_')[0] for col in field.colnames if col.endswith('flux_aper_1')]
filters1=filters1[1:]

filters2 = [col.split('_')[0] for col in field.colnames if col.endswith('flux_aper_2')]
filters2=filters2[1:]

filters3 = [col.split('_')[0] for col in field.colnames if col.endswith('flux_aper_3')]
filters3=filters3[1:]



In [ ]:

b_arr0 = [basic_lupt_soft(field[f'{filt}_flux_aper_0'], field[f'{filt}_fluxerr_aper_0']) for filt in filters0] 
b_arr1 = [basic_lupt_soft(field[f'{filt}_flux_aper_1'], field[f'{filt}_fluxerr_aper_1']) for filt in filters1] 
b_arr2 = [basic_lupt_soft(field[f'{filt}_flux_aper_2'], field[f'{filt}_fluxerr_aper_2']) for filt in filters2] 
b_arr3 = [basic_lupt_soft(field[f'{filt}_flux_aper_3'], field[f'{filt}_fluxerr_aper_3']) for filt in filters3] 

In [ ]:
i=15
lupt_cols=field['field','id','ra','dec']

for filt,b in zip(filters0,b_arr0):
    lupt,lupterr = flux_to_lupt(field[f'{filt}_flux_aper_0'],field[f'{filt}_fluxerr_aper_0'],b)
    lupt_cols[f'lupt_{filt}_aper0']=lupt
    lupt_cols[f'lupterr_{filt}_aper0']=lupterr
for filt,b in zip(filters1,b_arr1):
    lupt,lupterr = flux_to_lupt(field[f'{filt}_flux_aper_1'],field[f'{filt}_fluxerr_aper_1'],b)
    lupt_cols[f'lupt_{filt}_aper1']=lupt
    lupt_cols[f'lupterr_{filt}_aper1']=lupterr
for filt,b in zip(filters2,b_arr2):
    lupt,lupterr = flux_to_lupt(field[f'{filt}_flux_aper_2'],field[f'{filt}_fluxerr_aper_2'],b)
    lupt_cols[f'lupt_{filt}_aper2']=lupt
    lupt_cols[f'lupterr_{filt}_aper2']=lupterr
for filt,b in zip(filters3,b_arr3):
    lupt,lupterr = flux_to_lupt(field[f'{filt}_flux_aper_3'],field[f'{filt}_fluxerr_aper_3'],b)
    lupt_cols[f'lupt_{filt}_aper3']=lupt
    lupt_cols[f'lupterr_{filt}_aper3']=lupterr

lupt_cols.write(f'MPhys/catalogues/{fields[i]}_lupts.fits',format='fits', overwrite=True)

In [2]:
lup_ceers=Table.read('MPhys/catalogues/ceers-full_lupts.fits',format='fits')
lup_gds=Table.read('MPhys/catalogues/gds_lupts.fits',format='fits')
lup_gdn=Table.read('MPhys/catalogues/gdn_lupts.fits',format='fits')
lup_ngdeep=Table.read('MPhys/catalogues/ngdeep_lupts.fits',format='fits')
lup_udss=Table.read('MPhys/catalogues/primer-uds-south_lupts.fits',format='fits')
lup_udsn=Table.read('MPhys/catalogues/primer-uds-north_lupts.fits',format='fits')
lup_cosmos_east=Table.read('MPhys/catalogues/primer-cosmos-east_lupts.fits',format='fits')
lup_cosmos_west=Table.read('MPhys/catalogues/primer-cosmos-west_lupts.fits',format='fits')

In [3]:
lup_abell2744clu = Table.read('MPhys/catalogues/abell2744clu_lupts.fits',format='fits')
lup_abells1063=Table.read('MPhys/catalogues/abells1063_lupts.fits',format='fits')
lup_elgordo=Table.read('MPhys/catalogues/elgordo_lupts.fits',format='fits')
lup_j112716p4228=Table.read('MPhys/catalogues/j112716p4228_lupts.fits',format='fits')
lup_macs0647=Table.read('MPhys/catalogues/macs0647_lupts.fits',format='fits')
lup_rxj2129=Table.read('MPhys/catalogues/rxj2129_lupts.fits',format='fits')
lup_smacs0723=Table.read('MPhys/catalogues/smacs0723_lupts.fits',format='fits')
lup_sunrise=Table.read('MPhys/catalogues/sunrise_lupts.fits',format='fits')

In [4]:
#saves laptop dying if split up 
lup_con =vstack([lup_ceers,lup_gds,lup_gdn,lup_ngdeep,lup_udss,lup_udsn,lup_cosmos_east,lup_cosmos_west,])


In [5]:
lup_con_small = vstack([lup_abell2744clu,lup_abells1063,lup_elgordo,lup_j112716p4228,lup_macs0647,lup_rxj2129,lup_smacs0723,lup_sunrise])

In [6]:
lup_total = vstack([lup_con,lup_con_small])

In [7]:
#needs to be done in CP Lab 
lup_total.write('luptitudes_all_jwst.fits', format='fits', overwrite=True)


# Skymatching Catalogues

In [116]:
candels=Table.read('MPhys/CANDELS_dawn_catalogue.fits',format='fits')
jwst=Table.read('luptitudes_all_jwst.fits',format='fits')
dawn = Table.read('csv.csv',format='csv')

In [117]:
z = dawn['zspec']
z[z < 0] = np.nan
masked=[]
for i in range(len(dawn)):
    masked.append(np.ma.is_masked(z[i]))
masked_inv=np.array(np.invert(masked))
masked_cat =dawn[masked_inv*(np.ma.is_masked(z) == True)] 
nans=[]
for i in range(len(masked_cat)):
    nans.append(np.isnan(masked_cat['zspec'][i]))
nans_inv=np.array(np.invert(nans))
dawn_zreal =masked_cat[nans_inv*(np.isnan(masked_cat['zspec']) == False)] 

In [141]:
dawn_zreal

jname,count,F200W,F444W,ra,dec,id,zphot,zspec,arxiv,author,z_spec
str20,int64,int64,int64,float64,float64,str26,float64,float64,float64,str21,float64
j001418.52m302521.27,2,--,--,3.577166,-30.42258,GLASS-150029,--,4.584,2301.06811,Yuki Isobe,4.584
j001419.27m302527.85,2,--,--,3.580275,-30.4244,GLASS_160133,--,4.015,2301.12825,Kimihiko Nakajima,4.015
j001424.93m302256.05,4,--,--,3.603845,-30.38224,GLASS-100001,--,7.874,2301.06811,Yuki Isobe,7.874
j033225.43m274830.50,1,--,--,53.105962,-27.80847,10036017,--,2.016,2306.02467,Andrew Bunker,2.016
j033226.59m274749.42,1,--,--,53.110776,-27.79706,10037047,--,2.058,2306.02467,Andrew Bunker,2.058
j033226.60m274802.37,1,--,--,53.110847,-27.80066,10036834,--,2.616,2306.02467,Andrew Bunker,2.616
j033226.97m274901.86,1,--,--,53.112387,-27.81718,3753,--,1.768,2306.02467,Andrew Bunker,1.768
j033228.88m274628.79,1,--,--,53.120315,-27.77466,10012005,--,1.864,2306.02467,Andrew Bunker,1.864
j033229.07m274628.90,1,--,--,53.12114,-27.77469,10011974,--,2.225,2306.02467,Andrew Bunker,2.225


In [119]:
ra1=dawn_zreal['ra']
dec1=dawn_zreal['dec']
ra2=candels['RA']
dec2=candels['DEC']


In [120]:
dawn_cat = SkyCoord(ra=ra1*u.degree, dec=dec1*u.degree)#dawn
candels_cat = SkyCoord(ra=ra2*u.degree, dec=dec2*u.degree)#candels


In [121]:
max_sep = 0.2 * u.arcsec
idx, d2d, d3d = dawn_cat.match_to_catalog_sky(candels_cat)#catalog = candels information
idx_dawn, d2d_dawn, d3d_dawn = candels_cat.match_to_catalog_sky(dawn_cat)

In [122]:
id_candel=[] #ids of candels catalog 
for i in range(len(ra1)):
    if d2d[i]< max_sep:
        id_candel.append(idx[i])
id_dawn=[] #ids of jwst catalog 
for i in range(len(ra2)):
    if d2d_dawn[i]< max_sep:
        id_dawn.append(idx_dawn[i])


In [123]:
id_candels_new=list(set(id_candel))
id_dawn_new = list(set(id_dawn)) 
matched_candels = candels[id_candels_new]
matched_dawn= dawn_zreal[id_dawn_new ]

In [137]:
#candels.remove_rows(id_candel)
dawn_zreal.remove_rows(id_dawn)
candels

ID,RA,DEC,CLASS_STAR,FLAGS,EBV,z_spec
,,,"0 galaxy, 1 star","0 use, gt 1 no-use",,
int64,float64,float64,float64,int64,float64,float64
4,150.0562464,2.174318,0.83,0,0.01908,2.1049
26,150.0605723,2.1755824,0.03,0,0.01906,0.8088
47,150.0887223,2.1772836,0.1,0,0.01883,2.4167
53,150.0830862,2.1778659,0.0,0,0.01889,2.9802
62,150.0571343,2.1774997,0.0,0,0.01912,1.049
68,150.0702489,2.1781997,0.0,0,0.01902,0.937
87,150.1042142,2.1789081,0.02,1,0.01844,1.976
102,150.0547914,2.1782588,0.0,1,0.01915,0.5617


In [130]:
matched_candels['z_spec']=matched_dawn['zspec'] #update candels zspec to jwst zspec

In [146]:
#dawn_zreal.rename_column('ra', 'RA')
#dawn_zreal.rename_column('dec', 'DEC')
#dawn_zreal.rename_column('zspec','z_spec')

KeyError: 'Column dec does not exist'

In [152]:
zspec_cat = vstack([candels,matched_candels, dawn_zreal])
specz_conf = zspec_cat['RA','DEC','z_spec']
specz_conf.write('MPhys/catalogues/full_specz_sources.fits',format='fits')

In [153]:
specz = Table.read('MPhys/catalogues/full_specz_sources.fits',format='fits')

In [154]:
ra3 = jwst['ra']
dec3=jwst['dec']
ra4=specz['RA']
dec4=specz['DEC']

jwst_cat = SkyCoord(ra=ra3*u.degree, dec=dec3*u.degree)#jwst
specz_cat = SkyCoord(ra=ra4*u.degree, dec=dec4*u.degree)#jwst

In [155]:
idx_specz, d2d_specz, d3d_specz = jwst_cat.match_to_catalog_sky(specz_cat)#catalog = candels information
idx_jwst, d2d_jwst, d3d_jwst = specz_cat.match_to_catalog_sky(jwst_cat)

In [157]:
id_specz=[] #ids of candels catalog 
for i in range(len(ra3)):
    if d2d_specz[i]< max_sep:
        id_specz.append(idx_specz[i])
id_jwst=[] #ids of jwst catalog 
for i in range(len(ra4)):
    if d2d_jwst[i]< max_sep:
        id_jwst.append(idx_jwst[i])


In [162]:
idx_new = list(set(id_specz)) #list of non-repeating candels indexs
idx_new_jwst = list(set(id_jwst)) 
matched_specz = specz[idx_new]
matched_jwst = jwst[idx_new_jwst]
matched_specz.sort('RA') #makes them in same order
matched_jwst.sort('ra')


RA,DEC,z_spec
float64,float64,float64
3.5652,-30.39426,8.473
3.570643,-30.41464,11.671
3.577001,-30.41552,5.503
3.577166,-30.42258,4.584
3.580275,-30.4244,4.61
3.582321,-30.38765,4.243
3.596084,-30.38581,5.944
3.599716,-30.43189,4.884
3.60135,-30.37921,4.022


In [165]:
#matched_jwst.add_column(matched_specz['z_spec'], name='z_spec', index=-1)
matched_jwst.write('MPhys/catalogues/jwst_skymatched_specz.fits',format='fits')

# Filter Choices for Training Catalogue


In [3]:
train = Table.read('MPhys/catalogues/jwst_skymatched_specz.fits',format = 'fits')
train

field,id,ra,dec,lupt_f1000w_aper0,lupterr_f1000w_aper0,lupt_f105w_aper0,lupterr_f105w_aper0,lupt_f115w-clear_aper0,lupterr_f115w-clear_aper0,lupt_f125w_aper0,lupterr_f125w_aper0,lupt_f1280w_aper0,lupterr_f1280w_aper0,lupt_f140w_aper0,lupterr_f140w_aper0,lupt_f1500w_aper0,lupterr_f1500w_aper0,lupt_f150w-clear_aper0,lupterr_f150w-clear_aper0,lupt_f160w_aper0,lupterr_f160w_aper0,lupt_f1800w_aper0,lupterr_f1800w_aper0,lupt_f182m-clear_aper0,lupterr_f182m-clear_aper0,lupt_f200w-clear_aper0,lupterr_f200w-clear_aper0,lupt_f2100w_aper0,lupterr_f2100w_aper0,lupt_f210m-clear_aper0,lupterr_f210m-clear_aper0,lupt_f275wu_aper0,lupterr_f275wu_aper0,lupt_f277w-clear_aper0,lupterr_f277w-clear_aper0,lupt_f356w-clear_aper0,lupterr_f356w-clear_aper0,lupt_f410m-clear_aper0,lupterr_f410m-clear_aper0,lupt_f435w_aper0,lupterr_f435w_aper0,lupt_f444w-clear_aper0,lupterr_f444w-clear_aper0,lupt_f560w_aper0,lupterr_f560w_aper0,lupt_f606w_aper0,lupterr_f606w_aper0,lupt_f606wu_aper0,lupterr_f606wu_aper0,lupt_f770w_aper0,lupterr_f770w_aper0,lupt_f814w_aper0,lupterr_f814w_aper0,lupt_f1000w_aper1,lupterr_f1000w_aper1,lupt_f105w_aper1,lupterr_f105w_aper1,lupt_f115w-clear_aper1,lupterr_f115w-clear_aper1,lupt_f125w_aper1,lupterr_f125w_aper1,lupt_f1280w_aper1,lupterr_f1280w_aper1,lupt_f140w_aper1,lupterr_f140w_aper1,lupt_f1500w_aper1,lupterr_f1500w_aper1,lupt_f150w-clear_aper1,lupterr_f150w-clear_aper1,lupt_f160w_aper1,lupterr_f160w_aper1,lupt_f1800w_aper1,lupterr_f1800w_aper1,lupt_f182m-clear_aper1,lupterr_f182m-clear_aper1,lupt_f200w-clear_aper1,lupterr_f200w-clear_aper1,lupt_f2100w_aper1,lupterr_f2100w_aper1,lupt_f210m-clear_aper1,lupterr_f210m-clear_aper1,lupt_f275wu_aper1,lupterr_f275wu_aper1,lupt_f277w-clear_aper1,lupterr_f277w-clear_aper1,lupt_f356w-clear_aper1,lupterr_f356w-clear_aper1,lupt_f410m-clear_aper1,lupterr_f410m-clear_aper1,lupt_f435w_aper1,lupterr_f435w_aper1,lupt_f444w-clear_aper1,lupterr_f444w-clear_aper1,lupt_f560w_aper1,lupterr_f560w_aper1,lupt_f606w_aper1,lupterr_f606w_aper1,lupt_f606wu_aper1,lupterr_f606wu_aper1,lupt_f770w_aper1,lupterr_f770w_aper1,lupt_f814w_aper1,lupterr_f814w_aper1,lupt_f1000w_aper2,lupterr_f1000w_aper2,lupt_f105w_aper2,lupterr_f105w_aper2,lupt_f115w-clear_aper2,lupterr_f115w-clear_aper2,lupt_f125w_aper2,lupterr_f125w_aper2,lupt_f1280w_aper2,lupterr_f1280w_aper2,lupt_f140w_aper2,lupterr_f140w_aper2,lupt_f1500w_aper2,lupterr_f1500w_aper2,lupt_f150w-clear_aper2,lupterr_f150w-clear_aper2,lupt_f160w_aper2,lupterr_f160w_aper2,lupt_f1800w_aper2,lupterr_f1800w_aper2,lupt_f182m-clear_aper2,lupterr_f182m-clear_aper2,lupt_f200w-clear_aper2,lupterr_f200w-clear_aper2,lupt_f2100w_aper2,lupterr_f2100w_aper2,lupt_f210m-clear_aper2,lupterr_f210m-clear_aper2,lupt_f275wu_aper2,lupterr_f275wu_aper2,lupt_f277w-clear_aper2,lupterr_f277w-clear_aper2,lupt_f356w-clear_aper2,lupterr_f356w-clear_aper2,lupt_f410m-clear_aper2,lupterr_f410m-clear_aper2,lupt_f435w_aper2,lupterr_f435w_aper2,lupt_f444w-clear_aper2,lupterr_f444w-clear_aper2,lupt_f560w_aper2,lupterr_f560w_aper2,lupt_f606w_aper2,lupterr_f606w_aper2,lupt_f606wu_aper2,lupterr_f606wu_aper2,lupt_f770w_aper2,lupterr_f770w_aper2,lupt_f814w_aper2,lupterr_f814w_aper2,lupt_f1000w_aper3,lupterr_f1000w_aper3,lupt_f105w_aper3,lupterr_f105w_aper3,lupt_f115w-clear_aper3,lupterr_f115w-clear_aper3,lupt_f125w_aper3,lupterr_f125w_aper3,lupt_f1280w_aper3,lupterr_f1280w_aper3,lupt_f140w_aper3,lupterr_f140w_aper3,lupt_f1500w_aper3,lupterr_f1500w_aper3,lupt_f150w-clear_aper3,lupterr_f150w-clear_aper3,lupt_f160w_aper3,lupterr_f160w_aper3,lupt_f1800w_aper3,lupterr_f1800w_aper3,lupt_f182m-clear_aper3,lupterr_f182m-clear_aper3,lupt_f200w-clear_aper3,lupterr_f200w-clear_aper3,lupt_f2100w_aper3,lupterr_f2100w_aper3,lupt_f210m-clear_aper3,lupterr_f210m-clear_aper3,lupt_f275wu_aper3,lupterr_f275wu_aper3,lupt_f277w-clear_aper3,lupterr_f277w-clear_aper3,lupt_f356w-clear_aper3,lupterr_f356w-clear_aper3,lupt_f410m-clear_aper3,lupterr_f410m-clear_aper3,lupt_f435w_aper3,lupterr_f435w_aper3,lupt_f444w-clear_aper3,lupterr_f

In [105]:
#Full catalogue filter names 
filters_1 = [col.split('_')[1] for col in train.colnames if col.endswith('_aper1')]
#print(len(filters_1))
x= int(184/4)+4
#lup_total['lupt_f250m-clear_aper0']
print(filters_1)

['f1000w', 'f1000w', 'f105w', 'f105w', 'f115w-clear', 'f115w-clear', 'f125w', 'f125w', 'f1280w', 'f1280w', 'f140w', 'f140w', 'f1500w', 'f1500w', 'f150w-clear', 'f150w-clear', 'f160w', 'f160w', 'f1800w', 'f1800w', 'f182m-clear', 'f182m-clear', 'f200w-clear', 'f200w-clear', 'f2100w', 'f2100w', 'f210m-clear', 'f210m-clear', 'f275wu', 'f275wu', 'f277w-clear', 'f277w-clear', 'f356w-clear', 'f356w-clear', 'f410m-clear', 'f410m-clear', 'f435w', 'f435w', 'f444w-clear', 'f444w-clear', 'f560w', 'f560w', 'f606w', 'f606w', 'f606wu', 'f606wu', 'f770w', 'f770w', 'f814w', 'f814w', 'f090w-clear', 'f090w-clear', 'f335m-clear', 'f335m-clear', 'f350lpu', 'f350lpu', 'f475w', 'f475w', 'f775w', 'f775w', 'f850lp', 'f850lp', 'f110w', 'f110w', 'f336wu', 'f336wu', 'f850lpu', 'f850lpu', 'clearp-f356w', 'clearp-f356w', 'clearp-f444w', 'clearp-f444w', 'f115wn-clear', 'f115wn-clear', 'f150wn-clear', 'f150wn-clear', 'f200wn-clear', 'f200wn-clear', 'f225wu', 'f225wu', 'f250m-clear', 'f250m-clear', 'f300m-clear', 'f30

In [106]:
#### FILTER CHOICES
bc= ['f115w-clear','f200w-clear','f356w-clear','f444w-clear']
#swap f115w for f814w,f606w,f435w,f125w,f140w,f160w,f150w
c1 =['f814w','f200w-clear','f356w-clear','f444w-clear']
c2 =['f606w','f200w-clear','f356w-clear','f444w-clear']
c3 =['f435w','f200w-clear','f356w-clear','f444w-clear']
c4 =['f125w','f200w-clear','f356w-clear','f444w-clear']
c5 =['f140w','f200w-clear','f356w-clear','f444w-clear']
c6 =['f160w','f200w-clear','f356w-clear','f444w-clear']
c7 =['f150w-clear','f200w-clear','f356w-clear','f444w-clear']
#swap f200w for f210m - less sources so should be short cat
c8 =  ['f115w-clear','f210m-clear','f356w-clear','f444w-clear']
#swap f444w for f410m
c9= ['f115w-clear','f200w-clear','f356w-clear','f410m-clear']

In [118]:
######change this to make new catalogue of filters
aper=1
choice=c9
lup_train = train['field','id','ra','dec','z_spec']
lup_train[f'lupt_{choice[0]}']=train[f'lupt_{choice[0]}_aper{aper}']
lup_train[f'lupterr_{choice[0]}']=train[f'lupterr_{choice[0]}_aper{aper}']
lup_train[f'lupt_{choice[1]}']=train[f'lupt_{choice[1]}_aper{aper}']
lup_train[f'lupterr_{choice[1]}']=train[f'lupterr_{choice[1]}_aper{aper}']
lup_train[f'lupt_{choice[2]}']=train[f'lupt_{choice[2]}_aper{aper}']
lup_train[f'lupterr_{choice[2]}']=train[f'lupterr_{choice[2]}_aper{aper}']
lup_train[f'lupt_{choice[3]}']=train[f'lupt_{choice[3]}_aper{aper}']
lup_train[f'lupterr_{choice[3]}']=train[f'lupterr_{choice[3]}_aper{aper}']
#lup_train[f'lupt_{choice[4]}']=train[f'lupt_{choice[4]}_aper{aper}']
#lup_train[f'lupterr_{choice[4]}']=train[f'lupterr_{choice[4]}_aper{aper}']

In [119]:
#filts=['f115w','f200w','f277w','f356w','f444w']

In [120]:
f1=[]
for i in range(len(lup_train)):
    f1.append(np.ma.is_masked(lup_train[f'lupt_{choice[0]}'][i]))
f1_inv=np.array(np.invert(f1))
f1_cat =lup_train[f1_inv*(np.ma.is_masked(lup_train[f'lupt_{choice[0]}']) == True)] 

In [121]:
#f1_cat

In [122]:

f2=[]
for i in range(len(f1_cat)):
    f2.append(np.ma.is_masked(f1_cat[f'lupt_{choice[1]}'][i]))
f2_inv=np.array(np.invert(f2))
f2_cat =f1_cat[f2_inv*(np.ma.is_masked(f1_cat[f'lupt_{choice[1]}']) ==True)] 

In [123]:
#f2_cat

In [124]:

f3=[]
for i in range(len(f2_cat)):
    f3.append(np.ma.is_masked(f2_cat[f'lupt_{choice[2]}'][i]))
f3_inv=np.array(np.invert(f3))
f3_cat =f2_cat[f3_inv*(np.ma.is_masked(f2_cat[f'lupt_{choice[2]}']) == True)] 

In [125]:
#f3_cat

In [126]:

f4=[]
for i in range(len(f3_cat)):
    f4.append(np.ma.is_masked(f3_cat[f'lupt_{choice[3]}'][i]))
f4_inv=np.array(np.invert(f4))
f4_cat =f3_cat[f4_inv*(np.ma.is_masked(f3_cat[f'lupt_{choice[3]}']) == True)] #false idk why but works 


In [127]:
f4_cat

field,id,ra,dec,z_spec,lupt_f115w-clear,lupterr_f115w-clear,lupt_f200w-clear,lupterr_f200w-clear,lupt_f356w-clear,lupterr_f356w-clear,lupt_f410m-clear,lupterr_f410m-clear
bytes13,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
abells2744clu,12302,3.565197354609437,-30.394263533201045,8.473,26.16194597790983,0.019592463599442195,26.087478866574386,0.016533056036876964,25.517356347813802,0.008397173682703629,25.703250944689007,0.015451453275986354
abells2744clu,5747,3.570633264263966,-30.414636178977904,11.671,26.712502490965832,0.026049937208744024,26.598132352625665,0.020032956648043044,26.02128072200748,0.010687683799440994,26.35778778164204,0.020832106830599596
abells2744clu,5524,3.5769964722690366,-30.415529356392863,5.503,27.335212863258526,0.04227717564173052,27.355843147908764,0.03560965654963773,26.881237769347127,0.020068416472688436,26.5898773034237,0.024657382393121575
abells2744clu,3960,3.577146485385779,-30.422546074196482,4.584,26.747794276373764,0.026114336785139913,26.590892024782605,0.01935071676051102,26.054930654294015,0.010555599791693675,26.45665068906766,0.0222293794966642
abells2744clu,3573,3.580297174080497,-30.424408035991412,4.61,26.684343145692196,0.025110204173980785,26.147969527781203,0.013506975603923254,25.680336155924937,0.008119805154344363,26.458650434584392,0.022214372069493643
abells2744clu,14953,3.5823379484224374,-30.387656731206793,4.243,26.739867329282262,0.028634184695736836,26.64346323365239,0.022123844345237762,26.462772135342096,0.014710240368984042,26.90497576333816,0.04327610100795141
abells2744clu,15786,3.5960817966475918,-30.38581783405742,5.944,27.332434237029688,0.040919468419549014,27.05692740484776,0.02674398544924948,26.913538608189594,0.0189528871459329,27.22837649260014,0.046135628173958665
abells2744clu,3692,3.599734632393244,-30.431896825633363,4.884,25.488841174528638,0.01692145022582151,24.887280915477884,0.008044583085121335,24.670024427118193,0.005669710096122684,24.95775084138466,0.015635390193512753
abells2744clu,19190,3.601340105065028,-30.37920379453032,4.022,26.966941290705616,0.031501245437457526,26.57577281136027,0.01865752139995835,26.360904585984773,0.011849535885922712,26.625997264129087,0.026104490234700375


In [89]:
#USE IF USING 5TH FILTER
#f5=[]
#for i in range(len(f4_cat)):
#    f5.append(np.ma.is_masked(f4_cat[f'lupt_{choice[4]}'][i]))
#f5_inv=np.array(np.invert(f5))
#f5_cat =f4_cat[f5_inv*(np.ma.is_masked(f4_cat[f'lupt_{choice[4]}']) == True)] 

In [128]:
#f4_cat.write('MPhys/catalogues/base_filter_choice.fits',format = 'fits',overwrite=True)
f4_cat.write('MPhys/catalogues/filter_choice_f444w_to_f410m.cat',format = 'ascii.commented_header',overwrite=True)

In [131]:
onlyfiles = [f for f in listdir('MPhys/catalogues/base_filter_permutations') if isfile(join('MPhys/catalogues/base_filter_permutations', f))]

['filter_choice_f115w_to_f140w.cat', 'filter_choice_f200w_to_f210m.cat', 'filter_choice_f115w_to_f435w.cat', 'filter_choice_f115w_to_f125w.cat', 'filter_choice_f115w_to_f606w.cat', 'filter_choice_f115w_to_f814w.cat', 'filter_choice_f444w_to_f410m.cat', 'filter_choice_f115w_to_f150w.cat', 'filter_choice_f115w_to_f160w.cat']


In [134]:
for file in onlyfiles:
    t_full =Table.read(f'MPhys/catalogues/base_filter_permutations/{file}',format = 'ascii.commented_header')
    train_sample = int(len(t_full)*0.8)
    t = t_full[0:train_sample]
    p = t_full[train_sample:-1]
    t.write(f'MPhys/catalogues/base_filter_permutations/TRAIN_{file}',format = 'ascii.commented_header')
    p.write(f'MPhys/catalogues/base_filter_permutations/PRED_{file}',format = 'ascii.commented_header')

In [21]:
#IGNORE EVERYTHING ELSE - MOVING TO ANOTHER NOTEBOOK 

6277

In [ ]:
fit= Table.read('MPhys/jwst_parameter_files/jwst_nans_removed_lupts_specz_TRAIN.fits', format = 'fits')
train = fit['id_1','ra_1','dec_1','lupt_f115w','lupterr_f115w','lupt_f200w','lupterr_f200w','lupt_f277w','lupterr_f277w','lupt_f356w','lupterr_f356w','lupt_f444w','lupterr_f444w','z_spec']
train.write('MPhys/gpz_files/jwst_TRAIN_2.cat',format='ascii.commented_header',overwrite=True)

In [ ]:
train.sort('id_1')

In [ ]:
train_split=train[0:int(0.8*len(train))]
pred_split=train[int(0.8*len(train)):-1]

In [ ]:
train_split.write('MPhys/gpz_files/TRAIN_80_split.cat', format ='ascii.commented_header', overwrite=True)
pred_split.write('MPhys/gpz_files/PRED_20_split.cat', format ='ascii.commented_header', overwrite=True)

In [ ]:
pred_split

In [ ]:
ceers_new.write('ceers_tst.fits',format='fits',overwrite=True)

In [ ]:
filt='f115w-clear_flux_aper_1'
cat_ceers = ceers[f'{filt}','flux_auto','flux_aper_1']
cat_gds= gds[f'{filt}','flux_auto','flux_aper_1']
cat_gdn = gdn[f'{filt}','flux_auto','flux_aper_1']
cat_ngdeep = ngdeep[f'{filt}','flux_auto','flux_aper_1']
cat_cosmos_west = cosmos_west[f'{filt}','flux_auto','flux_aper_1']
cat_cosmos_east = cosmos_east[f'{filt}','flux_auto','flux_aper_1']
cat_udss = udss[f'{filt}','flux_auto','flux_aper_1']
cat_udsn = udsn[f'{filt}','flux_auto','flux_aper_1']


    


In [ ]:
flux_ceers=[]
mag_ceers=[]

flux_gds=[]
mag_gds=[]

flux_gdn=[]
#mag_gdn=[]

flux_ngdeep=[]
mag_ngdeep=[]

flux_cosmos_west=[]
mag_cosmos_west=[]

flux_udss=[]
mag_udss=[]

flux_udsn=[]
#mag_udsn=[]

flux_cosmos_east=[]
#mag_cosmos_east=[]

for j in range(len(cat_ceers)):
    flux_ceers.append((cat_ceers[f'{filt}'][j] * cat_ceers['flux_auto'][j]) /cat_ceers['flux_aper_1'][j])
    mag_ceers.append(23.9-(2.5*np.log10(flux_ceers[j])))
for j in range(len(cat_gds)):
    flux_gds.append((cat_gds[f'{filt}'][j] * cat_gds['flux_auto'][j]) /cat_gds['flux_aper_1'][j])
    mag_gds.append(23.9-(2.5*np.log10(flux_gds[j])))
for j in range(len(cat_gdn)):
    flux_gdn.append((cat_gdn[f'{filt}'][j] * cat_gdn['flux_auto'][j]) /cat_gdn['flux_aper_1'][j])
    mag_gds.append(23.9-(2.5*np.log10(flux_gdn[j])))
for j in range(len(cat_cosmos_west)):
    flux_cosmos_west.append((cat_cosmos_west[f'{filt}'][j] * cat_cosmos_west['flux_auto'][j]) /cat_cosmos_west['flux_aper_1'][j])
    mag_cosmos_west.append(23.9-(2.5*np.log10(flux_cosmos_west[j])))
for j in range(len(cat_cosmos_east)):
    flux_cosmos_east.append((cat_cosmos_east[f'{filt}'][j] * cat_cosmos_east['flux_auto'][j]) /cat_cosmos_east['flux_aper_1'][j])
    mag_cosmos_west.append(23.9-(2.5*np.log10(flux_cosmos_east[j])))
for j in range(len(cat_udss)):
    flux_udss.append((cat_udss[f'{filt}'][j] * cat_udss['flux_auto'][j]) /cat_udss['flux_aper_1'][j])
    mag_udss.append(23.9-(2.5*np.log10(flux_udss[j])))
for j in range(len(cat_udsn)):
    flux_udsn.append((cat_udsn[f'{filt}'][j] * cat_udsn['flux_auto'][j]) /cat_udsn['flux_aper_1'][j])
    mag_udss.append(23.9-(2.5*np.log10(flux_udsn[j])))
for j in range(len(cat_ngdeep)):
    flux_ngdeep.append((cat_ngdeep[f'{filt}'][j] * cat_ngdeep['flux_auto'][j]) /cat_ngdeep['flux_aper_1'][j])
    mag_ngdeep.append(23.9-(2.5*np.log10(flux_ngdeep[j])))


In [ ]:
plt.hist(mag_ceers,histtype='step',bins=100,label='CEERS')
plt.hist(mag_gds,histtype='step',bins=100,label='GDS')
plt.hist(mag_cosmos_west,histtype='step',bins=100,label='PRIMER COSMOS')
plt.hist(mag_udss,histtype='step',bins=100,label='UDS')
plt.hist(mag_ngdeep,histtype='step',bins=100,label='NGDeep')
plt.xlabel('AB Magnitudes')
plt.title(f'AB Magnitudes for {filt}')
plt.legend()
plt.show()
